In [1]:
import numpy as np
import pandas as pd
import pylab as pl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv ('Data_B_KNN.csv')
df.head ()

,Patient_Hospital,Outcome_InhospitalMortality,Outcome_LOS,Outcome_ICUadmission,Outcome_ICULOS,Demographic_Age,Demographic_Gender,Symptom_Caugh,Symptom_Dyspnea,Symptom_Fever,...,LAB_CR_1,LAB_NA_First,LAB_K_First,LAB_ALKP_First,LAB_ESR_First,LAB_CPK_First,LAB_PTT_First,LAB_PT_First,LAB_INR_First,LAB_BloodGroupABO_First
0,0,1,30.0,1,25.0,33.0,0,1,1.0,0,...,1.143188,140.0,4.0,181.0,62.0,661.0,32.6,10.9,1.00,9.000000
1,0,1,19.0,1,15.0,44.0,0,1,1.0,0,...,1.100000,139.0,3.5,98.0,30.0,179.0,37.6,10.8,0.99,21.542338
2,0,1,12.0,1,11.0,50.0,0,0,1.0,0,...,0.900000,138.0,4.2,99.0,29.0,1956.0,27.4,12.1,1.11,18.299813
3,0,0,18.0,1,9.0,71.0,0,0,1.0,1,...,1.200000,137.0,4.7,139.0,20.0,30.0,29.1,14.2,1.31,20.562657
4,0,0,18.0,1,9.0,71.0,0,0,1.0,1,...,1.200000,137.0,4.7,139.0,20.0,30.0,29.1,14.2,1.31,22.488693


In [3]:
from sklearn import preprocessing

Output_features = ['Outcome_InhospitalMortality','Outcome_ICUadmission','TM_S_Dialysis','TM_S_Intubation']

X = np.asarray (df.loc[:, ~df.columns.isin(Output_features)])
X = preprocessing.StandardScaler().fit(X).transform(X)
y = np.asarray (df [Output_features])


X = X.astype('float32')
y = y.astype('float32')

# Let's split our data into training and testing sets (adjust the test_size as needed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [4]:
forest_list = []


for i in range(len(Output_features)):
    forest_model = RandomForestClassifier (n_estimators=100, random_state = 4)
    forest_model.fit(X_train, y_train[:, i])
    importance = forest_model.feature_importances_
    forest_list.append(importance)

In [5]:
Column_names = df.loc[:, ~df.columns.isin(Output_features)].columns.tolist ()
feature_selection_df = pd.DataFrame({col_name: arr for col_name, arr in zip(Output_features, forest_list)},index = Column_names)

In [6]:
top_indices_df = pd.DataFrame(columns=feature_selection_df.columns)
top_n = 30

for col in feature_selection_df.columns:
    sorted_df = feature_selection_df.sort_values(by=col, ascending=False)
    # Append the top indices to the result DataFrame
    top_indices_df [col] = sorted_df.index[:top_n]


# Reset the index of the result DataFrame
top_indices_df.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame
print(top_indices_df)

# Reset the index of the result DataFrame
top_indices_df.reset_index(drop=True, inplace=True)

   Outcome_InhospitalMortality     Outcome_ICUadmission  \
0                LAB_BUN_Final           Outcome_ICULOS   
1               Outcome_ICULOS              Outcome_LOS   
2                 TM_fentanyl             LAB_BUN_Final   
3              Demographic_Age             TM_fentanyl    
4          VS_O2satwithoutsupp      VS_O2satwithoutsupp   
5                     TM_Vanco                 TM_Vanco   
6                     LAB_CR_1            LAB_ESR_First   
7                 LAB_LYMPHH_1             LAB_LYMPHH_1   
8                  Outcome_LOS                    VS_PR   
9                    LAB_WBC_1                 LAB_CR_1   
10                  LAB_NEUT_1            LAB_PTT_First   
11                       VS_PR                LAB_WBC_1   
12                 Symptom_LOC               LAB_NEUT_1   
13               LAB_INR_First             LAB_PT_First   
14                    LAB_HB_1            LAB_INR_First   
15                LAB_PT_First                LAB_PLT_1 

In [7]:
common_values = set(top_indices_df.iloc[:, 0])  # Initialize with the values of the first column

for col in top_indices_df.columns[1:]:
    common_values = common_values.intersection(top_indices_df[col])

# Convert the result to a list if needed
common_values = list(common_values)

# Print the common values
print(common_values)
len (common_values)

['VS_T', 'LAB_LYMPHH_1', 'Outcome_LOS', 'LAB_CR_1', 'VS_RR', 'LAB_PT_First', 'Outcome_ICULOS', 'LAB_WBC_1', 'LAB_HB_1', 'VS_diastolic BP', 'LAB_BUN_Final', 'TM_fentanyl ', 'LAB_NA_First', 'VS_PR', 'LAB_K_First', 'LAB_NEUT_1', 'LAB_PLT_1', 'LAB_MCV_1', 'VS_Systolic BP', 'LAB_ALKP_First', 'LAB_PTT_First', 'VS_O2satwithoutsupp', 'LAB_CPK_First', 'LAB_BloodGroupABO_First', 'LAB_ESR_First', 'Demographic_Age', 'LAB_INR_First']


27